<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/TOEIC_Part5_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer による自然言語処理

### 準備

関連ライブラリのインストール

In [ ]:
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q datasets

関連ライブラリのインポート

In [ ]:
import numpy as np
import torch
import textwrap
from transformers import T5Tokenizer, BertTokenizer
from transformers import AutoModelForCausalLM, RobertaForMaskedLM
from transformers import BertForPreTraining

## 1. 日本語モデルによる文章中の空欄埋め





### モデルのダウンロード

In [ ]:
# トーカナイザの設定
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading
# モデルの設定
model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")

### 問題文の作成

In [ ]:
# 原文
text = "4年に1度オリンピックは開かれる。"

# 文頭に [CLS] を付加
text = "[CLS]" + text + "[SEP]"

# トークン化
tokens = tokenizer.tokenize(text)
print(tokens)

# トークンにマスクをかける
masked_idx = 5
tokens[masked_idx] = tokenizer.mask_token
print(tokens)

### 穴埋め問題を解く

補充すべき単語の推定 (id)

In [ ]:
# トークンから単語 ID に変換
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

# テンソルに変換
token_tensor = torch.LongTensor([token_ids])

結果の表示

In [ ]:
# 場所の ID を与える
position_ids = list(range(0, token_tensor.size(1)))
position_id_tensor = torch.LongTensor([position_ids])

# マスクされたトークンの予測値（トップ 10）
with torch.no_grad():
    outputs = model(input_ids=token_tensor, position_ids=position_id_tensor)
    predictions = outputs[0][0, masked_idx].topk(10)

for i, index_t in enumerate(predictions.indices):
    index = index_t.item()
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)

## 2. 英語モデルによる TOEIC Part 5 の解答

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')

### 問題文の作成

問題文：text、選択肢：candidate

In [ ]:
text = "Customer reviews indicate that many modern mobile devices are often unnecessarily [MASK] ."
candidate = ["complication", "complicates", "complicate", "complicated"]

BERTに分かるように変換 (text --> tokens)

In [ ]:
tokens = tokenizer.tokenize(text)
print(tokens)

masked_index = tokens.index("[MASK]")
tokens = ["[CLS]"] + tokens + ["[SEP]"]

print(tokens)
print(masked_index)

### 解答の作成

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids = torch.tensor(ids).reshape(1,-1)  # バッチサイズ1の形に整形
predictions = model(ids)[0][0]
print(predictions)

In [ ]:
_, predicted_indexes = torch.topk(predictions[masked_index+1], k=1000)
predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_indexes.tolist())
# -> ['expensive', 'small', 'priced', 'used', ...
print(predicted_tokens)

In [ ]:
for i, v in enumerate(predicted_tokens):
    if v in candidate:
        print(i, v)
        break

### 課題
ここまでの一連の作業を関数 part5_solver(text, candidate) として実装せよ。

ただし、この関数は、問題文(text)、選択肢の単語(candidate)を入力とし、解答となる単語を出力するものとする。

In [ ]:
def part5_slover(text, candidate):
    # ここにプログラムを記述する
    return answer

### 関数の動作確認（Part 5 練習問題で試してみる）
公式サンプル問題 --> https://www.iibc-global.org/toeic/test/lr/about/format/sample05.html

第２問〜第５問（問題文：text2〜5、選択肢：candidate2〜5）

In [ ]:
text2 = "Jamal Nawzad has received top performance reviews [MASK] he joined the sales department two years ago ."
candidate2 = ["despite", "except", "since", "during"]
text3 = "Gyeon Corporation’s continuing education policy states that [MASK] learning new skills enhances creativity and focus ."
candidate3 = ["regular", "regularity", "regulate", "regularly"]
text4 = "Among [MASK] recognized at the company awards ceremony were senior business analyst Natalie Obi and sales associate Peter Comeau. ."
candidate4 = ["who", "whose", "they", "those"]
text5 = "All clothing sold in Develyn’s Boutique is made from natural materials and contains no [MASK] dyes ."
candidate5 = ["immediate", "synthetic", "reasonable", "assumed"]

解答の作成

In [ ]:
print("answer:", part5_slover(text2, candidate2))
print("answer:", part5_slover(text3, candidate3))
print("answer:", part5_slover(text4, candidate4))
print("answer:", part5_slover(text5, candidate5))